In [1]:
# !pip3 install transformers datasets accelerate

## 1. Load the NSMC Dataset

In [2]:
import numpy as np
import pandas as pd
from Korpora import Korpora

corpus = Korpora.load("nsmc")
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)

# train : val : test 를 6 : 2 : 2 로 분리
train_df, val_df, test_df = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

# 출력 확인
print(train_df.head(5).to_markdown())
print(f"train: {len(train_df)}, val: {len(val_df)}")
print(f"test: {len(test_df)}")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/n

## 2. Tokenizer

In [3]:
from transformers import AutoTokenizer

# tokenizer 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
print(f"n_vocab: {tokenizer.vocab_size}")

# 출력 확인
example_sent = "Hello, Pytorch!"
encoded_sent = tokenizer(example_sent, return_tensors='pt')["input_ids"]
print(f"encoded: {encoded_sent}")

n_vocab: 119547
encoded: tensor([[  101, 31178,   117,   153, 20246, 10667, 10269,   106,   102]])


## 3. Supervised Learning

In [4]:
EPOCHS = 5
BATCH_SIZE = 32
LR = 1e-5

### 3-1. Dataset

In [5]:
import torch
from torch.utils.data import Dataset

class NSMCDataset(Dataset):
    def __init__(self, tokenizer, data):
        self.encoded = tokenizer(text=data['text'].tolist(),
                                 truncation=True,
                                 padding="longest",
                                 return_tensors='pt')
        self.labels = data['label'].tolist()
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
        item['labels'] = torch.tensor([self.labels[idx]], dtype=torch.long)
        
        return item
        
# 출력 확인
train_dataset, val_dataset = NSMCDataset(tokenizer, train_df), NSMCDataset(tokenizer, val_df)
for key, val in train_dataset[0].items():
    print(f"{key}.shape: {val.shape}")

input_ids.shape: torch.Size([124])
token_type_ids.shape: torch.Size([124])
attention_mask.shape: torch.Size([124])
labels.shape: torch.Size([1])


/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}


### 3-2. Define the Network

In [6]:
from transformers import BertForSequenceClassification

net = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3-3. Trainer & Evaluate

In [7]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

2024-05-08 13:56:01.627238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 13:56:01.721321: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 13:56:02.137806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-05-08 13:56:02.137891: W tensorflow/compiler/xla/stream_executor/

In [9]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    output_dir = "./pth/bert_sl",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = LR,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCHS,
)

trainer = Trainer(
    model = net,
    args = train_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ho/.local/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
trainer.train()

/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.468172,0.784000
2,0.527000,0.441393,0.799250
3,0.385600,0.442182,0.808750
4,0.309900,0.457088,0.814000
5,0.309900,0.479612,0.817500


/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy constru

TrainOutput(global_step=1875, training_loss=0.3773196004231771, metrics={'train_runtime': 804.9623, 'train_samples_per_second': 74.538, 'train_steps_per_second': 2.329, 'total_flos': 3823332523200000.0, 'train_loss': 0.3773196004231771, 'epoch': 5.0})

### 3-4. Test

In [14]:
test_dataset = NSMCDataset(tokenizer, test_df)
best_model = "./pth/bert_sl/checkpoint-1500"
net = BertForSequenceClassification.from_pretrained(best_model)

trainer = Trainer(
    model=net,
    args=TrainingArguments(
        output_dir = './pth/',
        per_device_eval_batch_size=BATCH_SIZE
    ),
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ho/.local/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}


{'eval_loss': 0.44626468420028687, 'eval_accuracy': 0.81175, 'eval_runtime': 12.564, 'eval_samples_per_second': 318.369, 'eval_steps_per_second': 9.949}


## 4. Pretraining(Self-Supervised Learning)

In [24]:
EPOCHS = 3
BATCH_SIZE = 16

### 4-1. NSP를 위해 문장 Pair 생성

In [17]:
import random

def create_sent_pairs(df):
    sent_a = []
    sent_b = []
    labels = []

    for i in range(len(df) - 1):
        sent_a.append(df.iloc[i]['text'])
        sent_b.append(df.iloc[i + 1]['text'])
        labels.append(1)  # 연속 문장
        sent_a.append(df.iloc[i]['text'])
        sent_b.append(df.iloc[random.randint(0, len(df)-1)]['text'])
        labels.append(0)  # 무작위 문장

    return sent_a, sent_b, labels

sent_a, sent_b, labels = create_sent_pairs(df)
print(f"{sent_a[0]} / {sent_b[0]}")
print(f"label: {labels[0]}")

모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. / 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...
label: 1


### 4-2. Dataset & DataCollator

In [18]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, tokenizer, sent_a, sent_b, labels):
        self.encoded = tokenizer(sent_a, sent_b, truncation=True, padding=True, max_length=512)
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
        item['next_sentence_label'] = torch.tensor([self.labels[idx]], dtype=torch.long)
        
        return item

dataset = CustomDataset(tokenizer, sent_a, sent_b, labels)
for key, val in dataset[0].items():
    print(f"{key}.shape: {val.shape}")


input_ids.shape: torch.Size([213])
token_type_ids.shape: torch.Size([213])
attention_mask.shape: torch.Size([213])
next_sentence_label.shape: torch.Size([1])


In [19]:
def data_collator(batch):
    input_ids = torch.stack([item['input_ids'].squeeze(0) for item in batch])
    attention_mask = torch.stack([item['attention_mask'].squeeze(0) for item in batch])
    labels = torch.stack([item['input_ids'].squeeze(0) for item in batch])  # MLM을 위해 input_ids를 레이블로 사용
    next_sentence_label = torch.stack([item['next_sentence_label'].squeeze(0) for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'next_sentence_label': next_sentence_label
    }

### 4-3 Define the Network

In [20]:
from transformers import BertForPreTraining

net = BertForPreTraining.from_pretrained("bert-base-multilingual-cased")

### 4-4. Trainer

In [26]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    output_dir = "./pth/pretrained",
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    logging_dir = "./result/logs"
)

trainer = Trainer(
    model = net,
    args = train_args,
    train_dataset = dataset,
    data_collator = data_collator
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ho/.local/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
trainer.train()

Step,Training Loss
500,0.728900
1000,0.706800
1500,0.705000


KeyboardInterrupt: 

## 5. Fine Tuning

In [37]:
EPOCHS = 5
BATCH_SIZE = 32
LR = 1e-5

### 5-1. Load Pretrained Model

In [39]:
from transformers import BertConfig

weight_path = './pth/pretrained/checkpoint-1500'  # 사전 학습된 모델 경로
config = BertConfig.from_pretrained(weight_path)
config.num_labels = 2  # NSMC는 긍정/부정 두 가지 레이블을 가짐

### 5-2. Define the Classification Model

In [40]:
import torch.nn as nn
from transformers import BertForSequenceClassification

net = BertForSequenceClassification.from_pretrained(
    weight_path,
    config=config
)

net.classifier = nn.Linear(config.hidden_size, 2)
net.classifier.apply(net._init_weights)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./pth/pretrained/checkpoint-1500 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=2, bias=True)

### 5-3. Trainer

In [41]:
train_args = TrainingArguments(
    output_dir = "./pth/bert_ssl",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = LR,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCHS,
)

trainer = Trainer(
    model = net,
    args = train_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ho/.local/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
trainer.train()

/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.561620,0.713250
2,0.636700,0.476821,0.772750
3,0.472400,0.461534,0.796000
4,0.390600,0.442894,0.801000
5,0.390600,0.464871,0.804250


/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy constru

TrainOutput(global_step=1875, training_loss=0.46896168619791667, metrics={'train_runtime': 6572.122, 'train_samples_per_second': 9.129, 'train_steps_per_second': 0.285, 'total_flos': 3823332523200000.0, 'train_loss': 0.46896168619791667, 'epoch': 5.0})

### Test

In [44]:
best_model = "./pth/bert_ssl/checkpoint-1875"
net = BertForSequenceClassification.from_pretrained(best_model)

trainer = Trainer(
    model=net,
    args=TrainingArguments(
        output_dir = './pth/',
        per_device_eval_batch_size=BATCH_SIZE
    ),
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ho/.local/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [45]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

/tmp/ipykernel_61183/2424498407.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}


{'eval_loss': 0.4567607343196869, 'eval_accuracy': 0.797, 'eval_runtime': 32.6795, 'eval_samples_per_second': 122.401, 'eval_steps_per_second': 3.825}
